# EX3

A **Empresa Passarinho Ltda.** produz dois tipos de ração animal: **ração forte** e **ração light**.

## Custos de Produção
- **Ração forte:** R$ 6,00 por kg
- **Ração light:** R$ 10,00 por kg

## Requisitos de Produção
- A diferença entre as quantidades de ração produzidas deve ser **no máximo 2kg**, sendo que a **ração light** deve ser produzida em maior quantidade.
- A produção da **ração forte** está limitada em **5kg**.
- A produção da **ração light** está limitada em **6kg**.

## Mão de Obra
- Para produzir **1kg** de **ração forte**, é necessária **1 pessoa**.
- Para produzir **1kg** de **ração light**, são necessárias **2 pessoas**.
- A empresa tem disponível **pelo menos 1 funcionário** e pode utilizar quantas pessoas forem necessárias para a produção final.

## Utilização de Insumos
- São utilizados **2 insumos** na produção.
- **Ração forte:** 
  - 3 unidades do **insumo 1** por kg
  - 5 unidades do **insumo 2** por kg
- **Ração light:** 
  - 5 unidades do **insumo 1** por kg
  - 4 unidades do **insumo 2** por kg

## Regras de Utilização de Insumos
- Os insumos possuem **vencimentos curtos**.
- O mínimo de **insumo 1** que deve ser utilizado é **15kg**.
- O mínimo de **insumo 2** que deve ser utilizado é **20kg**.


###  Modelo Matemático

#### Variáveis 



p1 = quantidade do produto forte a ser produzida 

p2 = quantidade do produto light a ser produzida

#### Função Objetivo

$Min \quad Z = 6*p1+10*p2$

#### Restrições

- diferença entre quantidade de produtos

$p2 - p1 <=2$

$p2 - p1 > 0$

- Limite produção

$p1<=5$

$p2<=6$

- Material

$p1*3 + p2*5 >=15$

$p1*5 + p2*4 >=20$



In [2]:
import pyomo.environ as pyo
import time

In [3]:
# Criação do modelo
model = pyo.ConcreteModel("Passarinho")

In [4]:
# Tipos produto:
model.P = pyo.Set(initialize=["forte", "light"])

# Tipos material:
model.M = pyo.Set(initialize=["1", "2"])

In [5]:
# ------------------------------
# Parâmetros
# ------------------------------


# Custos 
custo = {"forte":6, "light":10}
model.c = pyo.Param(model.P,initialize=custo)

# Qtd mão de obra
mao_obra = {"forte":1, "light":2}
model.mao_de_obra = pyo.Param(model.P,initialize=mao_obra)

# Limite producao
limt = {"forte":(0,5), "light":(0,6)}
# model.limite = pyo.Param(model.P,initialize=limt)

# Material
qtd_material = {("forte","1"):3, ("light","1"):5,("forte","2"):5, ("light","2"):4}
model.q_material = pyo.Param(model.P,model.M,initialize=qtd_material)

#utilização material
ut = {"1":15, "2":20}
model.min_material = pyo.Param(model.M,initialize=ut)

# diferença material
model.diff_material = pyo.Param(initialize=2)

In [6]:
# ------------------------------
# Variáveis de Decisão
# ------------------------------
# quantidade da ração tobi e max
model.x = pyo.Var(model.P,domain=pyo.NonNegativeIntegers,bounds = limt)

In [7]:
# ------------------------------
# Função Objetivo
# ------------------------------
def objective_rule(model):
    fo = sum(
        model.x[i] * model.c[i]
        for i in model.P
    )

    return fo

model.obj = pyo.Objective(rule=objective_rule, sense=pyo.minimize)

In [8]:
# ------------------------------
# Restrições
# ------------------------------

# 1. Diff quantidade de produtos - upper
def up_meteria(model):
    return model.x['light'] - model.x['forte'] <= 2

model.re_materia_up= pyo.Constraint(rule=up_meteria)

#1. Diff quantidade de produtos - lower
def lw_meteria(model):
    return model.x['light'] - model.x['forte']  >= 0

model.re_materia_lw= pyo.Constraint(rule=lw_meteria)

# 2. Material
def disponibilidade_material(model,m):
    return sum(model.x[i] * model.q_material[i,m] for i in model.P) >= model.min_material[m]

model.re_material = pyo.Constraint(model.M,rule=disponibilidade_material)



In [9]:
# ------------------------------
# Escrita do Modelo em Arquivo
# ------------------------------
model.write("ex3.lp", io_options={"symbolic_solver_labels": True})

# ------------------------------
# Resolução
# ------------------------------
solver = pyo.SolverFactory("appsi_highs")
results = solver.solve(model, tee=True)

# Exibindo resultados
print("\nStatus do solver:", results.solver.status)
print("Condição de terminação:", results.solver.termination_condition)
valor_obj = pyo.value(model.obj)
formatted = f"{valor_obj:,.2f}"
formatted = formatted.replace(",", "X").replace(".", ",").replace("X", ".")
print("Valor da Função Objetivo: R$", formatted)



MIP  has 4 rows; 2 cols; 8 nonzeros; 2 integer variables (0 binary)
Coefficient ranges:
  Matrix [1e+00, 5e+00]
  Cost   [6e+00, 1e+01]
  Bound  [5e+00, 6e+00]
  RHS    [2e+00, 2e+01]
Presolving model
4 rows, 2 cols, 8 nonzeros  0s
3 rows, 2 cols, 6 nonzeros  0s
3 rows, 2 cols, 6 nonzeros  0s
Objective function is integral with scale 0.5

Solving MIP model with:
   3 rows
   2 cols (0 binary, 2 integer, 0 implied int., 0 continuous)
   6 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point; X => User solution

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

 u  

In [10]:
for i in model.P:    
    val = pyo.value(model.x[i])
    if val is not None and val > 1e-6:
        print(f"  Quantidade do produto {i} : {val:.2f}")

  Quantidade do produto forte : 2.00
  Quantidade do produto light : 3.00
